In [2]:
!pip install tensorflow
!pip install tf-slim
!pip install tensorflow-object-detection-api

     |████████████████████████████████| 352 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 577 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 81.8 MB/s eta 0:00:01
     |████████████████████████████████| 5.0 MB 91.4 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 71.6 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 748 kB 76.0 MB/s eta 0:00:01
     |████████████████████████████████| 587 kB 70.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 33.8 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 23.9 MB/s eta 0:00:01
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844488 sha256=6cd38dd4a4cc64baba80fb2062bf76b042ce639f36e528bf9fd9f72fb5baa76f
  Stored in directory: /home/studio-lab-user/

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import tarfile
import requests
import glob

# Helper function to download and extract PASCAL VOC 2007 dataset
def download_and_extract_voc2007():
    url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar"
    target_dir = "./VOC2007"
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print("Downloading PASCAL VOC 2007 dataset...")
        response = requests.get(url, stream=True)
        tar_file = os.path.join(target_dir, "VOC2007.tar")
        with open(tar_file, "wb") as f:
            f.write(response.raw.read())
        print("Extracting dataset...")
        with tarfile.open(tar_file, "r") as tar:
            tar.extractall(path=target_dir)
        os.remove(tar_file)
        print("Dataset downloaded and extracted.")
    else:
        print("PASCAL VOC 2007 dataset already exists.")
    return os.path.join(target_dir, "VOCdevkit", "VOC2007")

# Load and preprocess an image from the dataset
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = np.array(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]  # Add batch dimension
    return image, input_tensor

# Display the image with bounding boxes and labels
def display_image_with_boxes(image, boxes, class_names, scores, threshold=0.5):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    ax = plt.gca()

    for box, cls, score in zip(boxes, class_names, scores):
        if score >= threshold:
            ymin, xmin, ymax, xmax = box
            rect = plt.Rectangle(
                (xmin * image.shape[1], ymin * image.shape[0]),
                (xmax - xmin) * image.shape[1],
                (ymax - ymin) * image.shape[0],
                fill=False,
                color="red",
                linewidth=2,
            )
            ax.add_patch(rect)
            ax.text(
                xmin * image.shape[1],
                ymin * image.shape[0],
                f"{cls}: {score:.2f}",
                bbox=dict(facecolor="yellow", alpha=0.5),
                fontsize=10,
                color="black",
            )
    plt.axis("off")
    plt.show()

# Main function
def main():
    # Step 1: Download and prepare dataset
    voc_dir = download_and_extract_voc2007()
    
    # Dynamically select the first image in the directory
    image_files = glob.glob(os.path.join(voc_dir, "JPEGImages", "*.jpg"))
    if not image_files:
        raise FileNotFoundError("No images found in the VOC2007/JPEGImages directory.")
    sample_image_path = image_files[0]
    print(f"Using sample image: {sample_image_path}")

    # Step 2: Load TensorFlow Hub Faster R-CNN model
    print("Loading Faster R-CNN model from TensorFlow Hub...")
    model_handle = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
    detector = hub.load(model_handle)
    print("Model loaded successfully.")

    # Step 3: Load and preprocess image
    image, input_tensor = load_image(sample_image_path)

    # Step 4: Run inference
    print("Running inference...")
    detections = detector(input_tensor)

    # Extract detection outputs
    boxes = detections["detection_boxes"][0].numpy()
    class_indices = detections["detection_classes"][0].numpy().astype(int)
    scores = detections["detection_scores"][0].numpy()

    # Map class indices to VOC labels (ensuring correct range)
    voc_labels = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle",
                  "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
                  "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

    # Filter invalid indices
    class_names = [
        voc_labels[idx] if idx < len(voc_labels) else "unknown"
        for idx in class_indices
    ]

    # Step 5: Display results
    display_image_with_boxes(image, boxes, class_names, scores)

if __name__ == "__main__":
    main()

PASCAL VOC 2007 dataset already exists.
Using sample image: ./VOC2007/VOCdevkit/VOC2007/JPEGImages/000005.jpg
Loading Faster R-CNN model from TensorFlow Hub...
Model loaded successfully.
Running inference...
